<a href="https://colab.research.google.com/github/ManyaChdry/Probation-Projects-2020/blob/main/VER_MLPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VOICE EMOTION RECOGNITION**

---
## DSC Probation Project (Sept-Oct 2020)

using MLPClassifier

In [33]:
pip install librosa soundfile numpy sklearn pyaudio

  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-c0ariufj/pyaudio/setup.py'"'"'; __file__='"'"'/tmp/pip-install-c0ariufj/pyaudio/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-_rr4nzjl/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [34]:
pip install soundfile

In [35]:
import librosa
import soundfile
import os
import glob
import _pickle as pickle
import numpy as np

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Now we define a function, '***extract_feature***', which takes in four values:  '*file-name*',  '*mfcc*',  '*chroma*' and  '*mel*'.

- **mfcc **: Mel Frequency Cepstral Coefficient, represents the short term power spectrum of sound

- **chroma** : Pertains to the 12 different Pitch classes
- **mel** : Mel Spectrogram Frequency




In [37]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type = 'kaiser_fast')
    if chroma:
       stft=np.abs(librosa.stft(X))
       result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y = X, sr = sample_rate, n_mfcc = 40).T, axis = 0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S = stft, sr = sample_rate).T,axis = 0)
        result=np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr = sample_rate).T,axis = 0)
        result = np.hstack((result, mel))
    return result

In [38]:
# Emotions in the dataset
emotions={
          '01':'neutral',
          '02':'calm',
          '03':'happy',
          '04':'sad',
          '05':'angry',
          '06':'fearful',
          '07':'disgust',
          '08':'surprised',
         }
# Emotions to observe
observed_emotions = ['neutral','calm','happy','sad','angry','fearful','disgust','surprised']

In [39]:
def load_data(test_size=0.2):
            x,y=[],[]
            for file in glob.glob('/content/drive/My Drive/speech/Actor_*/*.wav'):
                file_name=os.path.basename(file)
                emotion=emotions[file_name.split("-")[2]]
                if emotion not in observed_emotions:
                    continue
                feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
                x.append(feature)
                y.append(emotion)
            return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

In [40]:
# Splitting the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [41]:
x_train

array([[-5.90712982e+02,  7.46899927e+01, -2.91916731e+00, ...,
         1.83884071e-06,  2.86085510e-07,  3.31972317e-08],
       [-4.48751467e+02,  5.66297785e+01, -1.79117803e+01, ...,
         1.19333032e-05,  2.10709016e-06,  1.88782791e-07],
       [-6.48976321e+02,  7.70115638e+01,  1.34594055e+00, ...,
         6.90492324e-08,  1.73962315e-08,  1.80400662e-09],
       ...,
       [-6.78000854e+02,  5.65455570e+01, -2.11965199e+00, ...,
         1.51538677e-06,  3.79247510e-07,  4.45862856e-08],
       [-3.54952943e+02,  2.82286023e+01, -4.85562212e+01, ...,
         4.72839007e-05,  8.68914800e-06,  8.06023166e-07],
       [-6.58780738e+02,  6.58860921e+01, -5.95752691e+00, ...,
         2.04532784e-07,  4.99403598e-08,  3.97221845e-09]])

In [42]:
# Getting the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(1087, 363)


In [43]:
# Getting the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [44]:
# Initializing the Multi-Layer Perceptron Classifier
model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (300,), learning_rate = 'adaptive', max_iter = 500)

In [45]:
# Training the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [46]:
# Prediction of the test set
y_pred=model.predict(x_test)

In [47]:
# Calculating the accuracy of our model
accuracy = accuracy_score(y_true = y_test, y_pred = y_pred)

In [48]:
# Printing the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 56.20%
